<a href="https://colab.research.google.com/github/navyakarna/PYTORCH_CV/blob/main/AlexNet_on_HUman_activity_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

8 layers ,
image size it could process is larger - 227X227
pooling - maxpooling
padding - yes
activation function - RELU
Dropout - 50%

In [ ]:
import os
import torch
from torchvision import transforms , datasets
from torch.utils.data import DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

In [ ]:
from zipfile import ZipFile

# Specify the path to your zip file
zip_file_path = '/content/Archive.zip'

# Specify the directory where you want to extract the contents
extract_to_path = '/content/'

# Open the zip file
with ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all contents to the specified directory
    zip_ref.extractall(extract_to_path)

print(f"Extracted files to {extract_to_path}")

Extracted files to /content/


In [ ]:
#paths to the data

train_data_path = r'/content/Human Action Recognition/test'
test_data_path = r'/content/Human Action Recognition/test'

In [ ]:
#apply the transforms to the train and test data
train_data = datasets.ImageFolder(root=train_data_path, transform=transforms)
test_data = datasets.ImageFolder(root=test_data_path, transform=transforms)

In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

class AlexNet(nn.Module):
  def __init__(self, num_classes=5):
    super(AlexNet, self).__init__()  # Add super().__init__() call
    self.features = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2), # Corrected kernel_sie to kernel_size
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(64, 192, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(192, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2)
    )


    self.fc_input_dim = 256*6*6 #calaculate the size of the feature map after the last ReLu function

    self.classifier = nn.Sequential(
        nn.Dropout(p=0.5, inplace=False),
        nn.Linear(self.fc_input_dim, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),


    )



  def forward(self, x): # Corrected indentation for forward method
      x = self.features(x)
      x = x.view(x.size(0), -1)
      x = self.classifier(x)
      return x





In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cpu')

In [ ]:
import torch.optim as optim
model = AlexNet()

#define loss function and optimizers
criterion = nn.CrossEntropyLoss()
optimier = optim.Adam(model.parameters())

In [ ]:
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

# Initialize the lists to store train and test loss for each epoch
train_losses = []
test_losses = []

# Train the model
num_epochs = 20
best_loss = torch.inf
patience = 5
epochs_since_best = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # For models that return auxiliary outputs
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)  # Store the train loss for this epoch
    train_accuracy = 100. * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Evaluate on the test set
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # For models that return auxiliary outputs
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss /= len(test_loader)
    test_losses.append(test_loss)  # Store the test loss for this epoch
    test_accuracy = 100. * correct / total

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

    # Check for best accuracy and stop if not improved after five more epochs
    if test_loss < best_loss:
        best_loss = test_loss
        epochs_since_best = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the model
        print(f'Updated best model with accuracy: {test_accuracy:.2f}%')
    else:
        epochs_since_best += 1
        if epochs_since_best > patience:
            print("Stopping early: no improvement after five consecutive epochs.")
            break
